In [1]:
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import HTML, display, JSON

import unstructured_client
from unstructured_client import UnstructuredClient
from unstructured_client.models import operations, shared
from unstructured_client.models.errors import SDKError

from unstructured.partition.html import partition_html
from unstructured.staging.base import dict_to_elements, elements_to_json


import json

In [25]:
client = unstructured_client.UnstructuredClient(
    api_key_auth="3gMFo57kcVKZd4dvct8ax6cTQk206a",
    server_url="https://api.unstructuredapp.io",
)

In [47]:
conn = sqlite3.connect('investopedia_test.db')
query = "SELECT * FROM investopedia LIMIT 1"

df_first_record = pd.read_sql_query(query, conn)


# Display the first record
df_first_record

,id,url,article_pre_content,article_content,article_post_content,processed_json
0,1,https://www.investopedia.com/terms/1/0x-protoc...,"<div class=""loc article-pre-content""><ul class...","<div class=""loc article-content""><div class=""c...","<div class=""loc article-post-content""><div cla...",None


In [39]:
article_pre_content_html = df_first_record.loc[0, 'article_pre_content']
article_content_html = df_first_record.loc[0, 'article_content']
article_post_content_html = df_first_record.loc[0, 'article_post_content']

In [2]:
def extract_first_title_element(elements):
    """
    Extracts and returns the first element of type 'Title' from a list of elements.
    
    Parameters:
    elements (list): List of element objects to search through.
    
    Returns:
    list: A list containing the first 'Title' element if found, otherwise an empty list.
    """
    for el in elements:
        if el.__class__.__name__ == 'Title':
            return [el]
    return []

In [75]:
elements_pre_content = partition_html(text=article_pre_content_html)

# Extract the first 'Title' element from the pre-content
title_pre_content = extract_first_title_element(elements_pre_content)
title_pre_content

In [46]:
elements_pre_content = partition_html(text=article_pre_content_html)
elements_content = partition_html(text=article_content_html)
elements_post_content = partition_html(text=article_post_content_html)

element_dict = [el.to_dict() for el in elements_pre_content + elements_content + elements_post_content]

processed_json = json.dumps(element_dict, indent=2)

conn = sqlite3.connect('investopedia_test.db')

cursor = conn.cursor()
update_query = "UPDATE investopedia SET processed_json = ? WHERE id = ?"
cursor.execute(update_query, (processed_json, df_first_record.loc[0, 'id']))

# Commit changes and close the connection
conn.commit()

# Read the updated record
query = "SELECT * FROM investopedia LIMIT 1"
df_updated_record = pd.read_sql_query(query, conn)

# Display the updated record

conn.close()

df_updated_record

,id,url,article_pre_content,article_content,article_post_content,processed_json
0,1,https://www.investopedia.com/terms/1/0x-protoc...,"<div class=""loc article-pre-content""><ul class...","<div class=""loc article-content""><div class=""c...","<div class=""loc article-post-content""><div cla...",None


In [3]:
conn = sqlite3.connect('investopedia_cleaned.db')
cursor = conn.cursor()

# Add a new column to store JSON data
cursor.execute("ALTER TABLE investopedia ADD COLUMN processed_json TEXT")

# Commit changes and close the connection
conn.commit()
conn.close()

In [ ]:
# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('investopedia_cleaned.db')

# Create a cursor object
cursor = conn.cursor()

# Create table to store the investopedia
cursor.execute('''
CREATE TABLE IF NOT EXISTS cleaned (
    id INTEGER PRIMARY KEY,
    title TEXT,
    article_pre_content TEXT,
    article_content TEXT,
    article_post_content TEXT
)
''')

# Commit changes and close the connection for now
conn.commit()
conn.close()

In [10]:
pre_content_html = df_first_record.loc[0, 'article_pre_content']
url = df_first_record.loc[0, 'url']
print(url)
soup_pre_content = BeautifulSoup(pre_content_html, 'html.parser') if pre_content_html else None
if soup_pre_content:
    # extract the H1 tag
    h1 = soup_pre_content.h1
    print(h1)
    #extract the content of the H1 tag
    h1_content = h1.text if h1 else None
    print(h1_content)

https://www.investopedia.com/terms/1/0x-protocol.asp
<h1 class="comp article-heading mntl-text-block" id="article-heading_1-0">0x (ZRX) Protocol: What It Meant and How It Worked</h1>
0x (ZRX) Protocol: What It Meant and How It Worked


In [49]:
df_first_record.loc[0, 'id']

np.int64(1)

In [50]:
# Step 1: Add a new column to the database (if it doesn't already exist)
def add_json_column_if_not_exists(conn):
    cursor = conn.cursor()
    cursor.execute("PRAGMA table_info(investopedia)")
    columns = [column[1] for column in cursor.fetchall()]
    if 'processed_json' not in columns:
        cursor.execute("ALTER TABLE investopedia ADD COLUMN processed_json TEXT")
    conn.commit()

# Connect to the SQLite database
conn = sqlite3.connect('investopedia_test.db')
add_json_column_if_not_exists(conn)

# Step 2: Load the first record into a pandas DataFrame
query = "SELECT * FROM investopedia LIMIT 1"
df_first_record = pd.read_sql_query(query, conn)

# Display the first record
print("First Record:\n", df_first_record)

article_pre_content_html = df_first_record.loc[0, 'article_pre_content']
article_content_html = df_first_record.loc[0, 'article_content']
article_post_content_html = df_first_record.loc[0, 'article_post_content']

# Step 3: Process the HTML content
elements_pre_content = partition_html(text=article_pre_content_html)
elements_content = partition_html(text=article_content_html)
elements_post_content = partition_html(text=article_post_content_html)

element_dict = [el.to_dict() for el in elements_pre_content + elements_content + elements_post_content]

# Convert the element dictionary to JSON
processed_json = json.dumps(element_dict, indent=2)

# Print the processed JSON
print("Processed JSON:\n", processed_json)

# Step 4: Store the processed JSON data in the database
cursor = conn.cursor()
update_query = "UPDATE investopedia SET processed_json = ? WHERE id = ?"
cursor.execute(update_query, (processed_json, int(df_first_record.loc[0, 'id'])))

# Commit changes
conn.commit()

# Read the updated record
query = "SELECT * FROM investopedia WHERE id = ?"

df_updated_record = pd.read_sql_query(query, conn, params=(int(df_first_record.loc[0, 'id']),))

# Display the updated record
print("Updated Record:\n", df_updated_record)

# Close the connection
conn.close()


First Record:
    id                                                url  \
0   1  https://www.investopedia.com/terms/1/0x-protoc...   

                                 article_pre_content  \
0  <div class="loc article-pre-content"><ul class...   

                                     article_content  \
0  <div class="loc article-content"><div class="c...   

                                article_post_content processed_json  
0  <div class="loc article-post-content"><div cla...           None  
Processed JSON:
 [
  {
    "type": "ListItem",
    "element_id": "e5a4c47939cfbe8b882cce977db657d1",
    "text": "Cryptocurrency",
    "metadata": {
      "category_depth": 1,
      "emphasized_text_contents": [
        "Cryptocurrency"
      ],
      "emphasized_text_tags": [
        "span"
      ],
      "link_texts": [
        ""
      ],
      "link_urls": [
        "https://www.investopedia.com/cryptocurrency-4427699"
      ],
      "link_start_indexes": [
        0
      ],
      "langua

In [4]:
# Connect to the SQLite database
conn = sqlite3.connect('investopedia_cleaned.db')

# Step 2: Load all records into a pandas DataFrame
query = "SELECT * FROM investopedia"
df_all_records = pd.read_sql_query(query, conn)

# Step 3: Process the HTML content for each record
def process_html_content(row):
    article_pre_content_html = row['article_pre_content']
    article_content_html = row['article_content']
    article_post_content_html = row['article_post_content']

    elements_pre_content = partition_html(text=article_pre_content_html)
    elements_content = partition_html(text=article_content_html)
    elements_post_content = partition_html(text=article_post_content_html)

    element_dict = [el.to_dict() for el in extract_first_title_element(elements_pre_content) + elements_content + elements_post_content]

    # Convert the element dictionary to JSON
    processed_json = json.dumps(element_dict, indent=2)
    return processed_json

# Update each record with processed JSON
cursor = conn.cursor()
update_query = "UPDATE investopedia SET processed_json = ? WHERE id = ?"

for index, row in df_all_records.iterrows():
    processed_json = process_html_content(row)
    cursor.execute(update_query, (processed_json, int(row['id'])))

# Commit all changes
conn.commit()

# Read the updated records
df_updated_records = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Display the updated records (optional, for verification)
print("Updated Records:\n", df_updated_records)


Updated Records:
         id                                                url  \
0        1  https://www.investopedia.com/terms/1/0x-protoc...   
1        2  https://www.investopedia.com/terms/1/1-10net30...   
2        3      https://www.investopedia.com/terms/1/10-k.asp   
3        4  https://www.investopedia.com/terms/1/10k-wrap.asp   
4        5       https://www.investopedia.com/terms/1/10q.asp   
...    ...                                                ...   
6283  6284  https://www.investopedia.com/terms/z/zone-of-r...   
6284  6285  https://www.investopedia.com/terms/z/zone-of-s...   
6285  6286    https://www.investopedia.com/terms/z/zoning.asp   
6286  6287  https://www.investopedia.com/terms/z/zoning-or...   
6287  6288  https://www.investopedia.com/terms/z/zzzzbest.asp   

                                    article_pre_content  \
0     <div class="loc article-pre-content"><ul class...   
1     <div class="loc article-pre-content"><ul class...   
2     <div class="loc ar

In [61]:
df_updated_records['processed_json'][0]

'[\n  {\n    "type": "ListItem",\n    "element_id": "e5a4c47939cfbe8b882cce977db657d1",\n    "text": "Cryptocurrency",\n    "metadata": {\n      "category_depth": 1,\n      "emphasized_text_contents": [\n        "Cryptocurrency"\n      ],\n      "emphasized_text_tags": [\n        "span"\n      ],\n      "link_texts": [\n        ""\n      ],\n      "link_urls": [\n        "https://www.investopedia.com/cryptocurrency-4427699"\n      ],\n      "link_start_indexes": [\n        0\n      ],\n      "languages": [\n        "eng"\n      ],\n      "filetype": "text/html"\n    }\n  },\n  {\n    "type": "ListItem",\n    "element_id": "555bd2e9aa59de6f3b7d17d671a5fac9",\n    "text": "Blockchain",\n    "metadata": {\n      "category_depth": 1,\n      "emphasized_text_contents": [\n        "Blockchain"\n      ],\n      "emphasized_text_tags": [\n        "span"\n      ],\n      "link_texts": [\n        ""\n      ],\n      "link_urls": [\n        "https://www.investopedia.com/blockchain-4689765"\n     